# Hiking Trail Time predictor

Goal: Use a database of hiking trails with various parameters to predict the average walking time. 

## Table of contents
1. [Introduction](#intro)
2. [Dataset](#dataset)
3. [Loading modules and data](#loading)
4. [Data pre-processing](#pre-process)


# Introduction <a name="intro"></a>

# Dataset <a name="dataset"></a>

# Loading modules and data <a name="loading"></a>

# Data pre-processing <a name="pre-process"></a>